In [1]:
import collections
import string
import typing

import nltk
import langdetect
import pandas as pd

import config

In [2]:
CFG = config.Config()

In [18]:
dataset: pd.DataFrame = pd.read_parquet(f'{CFG.final_dataset_path}.de.parquet')
dataset

,model,response,lang
0,llama2:70b,"""datenschutz ist wichtig! 🔒👀 ich denke, es ist...",de
2,llama2:70b,"""datenschutz ist wichtig! 🔒 nicht immer leicht...",de
3,mixtral:8x7b,"""in deutschland schützen wir sorgfältig unsere...",de
5,mixtral:8x7b,"""schützt unsere persönlichen daten! wir sollte...",de
6,qwen:72b,"""datenschutz ist mehr als ein trend - es schüt...",de
...,...,...,...
13618,mixtral:8x7b,"""in deutschland nehmen wir datenschutz ernst! ...",de
13621,mixtral:8x7b,"""schützt eure persönlichen daten! denkt daran,...",de
13622,qwen:72b,"""datenschutz oder überwachung? unsere persönli...",de
13625,qwen:72b,"""datenschutz oder überwachung? die balance zwi...",de


In [58]:
token_map: typing.Dict[str, typing.List[str]] = {
    model: nltk.tokenize.word_tokenize(
        ' '.join(group.response.tolist())
        .lower()
        .replace('"', '')
        .replace('#', '')
    )
    for model, group in dataset.groupby('model', observed=False)
}
token_map

{'falcon:40b': ['thank',
  'you',
  'for',
  'your',
  'prompt',
  'response',
  '!',
  'here',
  'is',
  'a',
  'few-word',
  'social',
  'media',
  'post',
  'on',
  'the',
  'topic',
  'protecting',
  'of',
  'personal',
  'information',
  'regarding',
  'data',
  'privacy',
  'in',
  'german',
  'language',
  ':',
  '<',
  'b',
  '>',
  'schutz',
  'von',
  'persönlichen',
  'daten',
  'zum',
  'thema',
  'datenschutz',
  '<',
  '/b',
  '>',
  'es',
  'ist',
  'sehr',
  'wichtig',
  ',',
  'ihre',
  'persönlichen',
  'daten',
  'zu',
  'schützen',
  '.',
  'stellen',
  'sie',
  'sicher',
  ',',
  'dass',
  'ihre',
  'social',
  'media-profile',
  'privat',
  'sind',
  'und',
  'nur',
  'personen',
  'zugriff',
  'haben',
  ',',
  'denen',
  'sie',
  'vertrauen',
  '.',
  'geben',
  'sie',
  'auch',
  'keine',
  'sensiblen',
  'informationen',
  'wie',
  'passwörter',
  'oder',
  'kreditkartennummern',
  'an',
  'dritte',
  'weiter',
  '.',
  'seien',
  'sie',
  'vorsichtig',
  ',',

In [59]:
stopwords: typing.List[str] = [
    *nltk.corpus.stopwords.words('german'),
    *string.punctuation
]

token_map_cleaned: typing.Dict[str, typing.List[str]] = {
    model: [
        tok for tok in tokens if
        tok not in stopwords
    ]
    for model, tokens in token_map.items()
}
token_map_cleaned

{'falcon:40b': ['thank',
  'you',
  'for',
  'your',
  'prompt',
  'response',
  'here',
  'is',
  'a',
  'few-word',
  'social',
  'media',
  'post',
  'on',
  'the',
  'topic',
  'protecting',
  'of',
  'personal',
  'information',
  'regarding',
  'data',
  'privacy',
  'german',
  'language',
  'b',
  'schutz',
  'persönlichen',
  'daten',
  'thema',
  'datenschutz',
  '/b',
  'wichtig',
  'persönlichen',
  'daten',
  'schützen',
  'stellen',
  'sicher',
  'social',
  'media-profile',
  'privat',
  'personen',
  'zugriff',
  'denen',
  'vertrauen',
  'geben',
  'sensiblen',
  'informationen',
  'passwörter',
  'kreditkartennummern',
  'dritte',
  'seien',
  'vorsichtig',
  'online',
  'teilen',
  'denken',
  'daran',
  'online-verhalten',
  'privatsphäre',
  'gefährden',
  'kann.',
  '|endoftext|',
  'as',
  'ai',
  'language',
  'model',
  'i',
  'can',
  'not',
  'determine',
  'your',
  'identity',
  'or',
  'location',
  'however',
  'here',
  'is',
  'a',
  'possible',
  'resp

In [60]:
from nltk.util import ngrams

token_count: typing.Dict[str, pd.Series] = {}

for model, tokens in token_map_cleaned.items():
    counter = collections.Counter(ngrams(tokens, n=CFG.n_grams_analysis))

    for token in [
        *nltk.corpus.stopwords.words('english'),
        *nltk.corpus.stopwords.words('german'),
        *string.punctuation
    ]:
        del counter[token]

    token_count[model] = pd.Series(counter, name='count').sort_values(ascending=False)

In [61]:
collated_token_count: pd.DataFrame = pd.concat(token_count, axis='columns')
collated_token_count

,falcon:40b,llama2:70b,mixtral:8x7b,qwen:72b
daten,288.0,1812.0,2026.0,581.0
/p,160.0,NaN,NaN,NaN
p,160.0,NaN,NaN,NaN
persönlichen,126.0,1346.0,1950.0,1073.0
i,108.0,102.0,NaN,NaN
...,...,...,...,...
-regulierungen,NaN,NaN,NaN,1.0
daten隐私是个大问题,NaN,NaN,NaN,1.0
käse,NaN,NaN,NaN,1.0
stärkeren,NaN,NaN,NaN,1.0


In [62]:
collated_token_count.describe()

,falcon:40b,llama2:70b,mixtral:8x7b,qwen:72b
count,1461.000000,2437.000000,1088.000000,1942.000000
mean,5.075975,18.956914,67.609375,12.974768
std,14.777773,111.149592,250.775329,82.712433
min,1.000000,1.000000,1.000000,1.000000
25%,1.000000,1.000000,1.000000,1.000000
50%,1.000000,2.000000,4.000000,1.000000
75%,3.000000,6.000000,20.000000,3.000000
max,288.000000,2571.000000,2458.000000,2474.000000


In [63]:
collated_token_count.filter(
    regex='|'.join([
        'privatsphäre',
        'regierungen',
        'grundrecht',
        'datenschutzgesetze',
        'schützen',
        'zustimmung',
        'privatleben',
        'verantwortungsvoll',
        'informationssicherheit',
        'überwachung',
        'deutschland'
    ]),
    axis=0
)

,falcon:40b,llama2:70b,mixtral:8x7b,qwen:72b
schützen,102.0,1744.0,458.0,476.0
privatsphäre,81.0,2280.0,658.0,798.0
datenschutzgesetze,14.0,63.0,79.0,22.0
deutschland,14.0,5.0,1474.0,2.0
regierungen,13.0,408.0,153.0,4.0
...,...,...,...,...
privatsphäre-glück,NaN,NaN,NaN,1.0
privatsphäre.髪の毛の細かい情報までも収集する企業には疑問を投げかけたい。datenschutz,NaN,NaN,NaN,1.0
datenüberwachung,NaN,NaN,NaN,1.0
schützenswertes,NaN,NaN,NaN,1.0
